In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [2]:
complete_df = pd.read_csv('complete_df.csv')

In [3]:
complete_df = complete_df.drop(['Unnamed: 0'], axis = 1)

In [4]:
complete_df.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'data_id',
       'dcoilwtico', 'city', 'state', 'type', 'cluster', 'transactions',
       'month', 'day', 'year', 'was_holiday', 'sale_cat'],
      dtype='object')

In [5]:
columns = [ 'sales', 'onpromotion',
       'dcoilwtico', 'city', 'state', 'type', 'cluster', 'transactions',
       'month', 'day', 'year', 'was_holiday', 'sale_cat']
pre_model_df = complete_df[columns]


In [6]:
pre_model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3029400 entries, 0 to 3029399
Data columns (total 13 columns):
 #   Column        Dtype  
---  ------        -----  
 0   sales         float64
 1   onpromotion   int64  
 2   dcoilwtico    float64
 3   city          object 
 4   state         object 
 5   type          object 
 6   cluster       int64  
 7   transactions  float64
 8   month         int64  
 9   day           int64  
 10  year          int64  
 11  was_holiday   bool   
 12  sale_cat      object 
dtypes: bool(1), float64(3), int64(5), object(4)
memory usage: 280.2+ MB


In [7]:
pre_model_df['was_holiday'] = pre_model_df['was_holiday'].astype('object')

/var/folders/0z/cp95yyc12mdglmpzxtzh5lv40000gn/T/ipykernel_4452/1856530797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_model_df['was_holiday'] = pre_model_df['was_holiday'].astype('object')


In [8]:
pre_model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3029400 entries, 0 to 3029399
Data columns (total 13 columns):
 #   Column        Dtype  
---  ------        -----  
 0   sales         float64
 1   onpromotion   int64  
 2   dcoilwtico    float64
 3   city          object 
 4   state         object 
 5   type          object 
 6   cluster       int64  
 7   transactions  float64
 8   month         int64  
 9   day           int64  
 10  year          int64  
 11  was_holiday   object 
 12  sale_cat      object 
dtypes: float64(3), int64(5), object(5)
memory usage: 300.5+ MB


In [9]:
target = pre_model_df['sales']
features = pre_model_df.drop('sales', axis = 1)

In [10]:
# select columns to encode or scale
cat_df = features.select_dtypes(include = ['object'])
num_df = features.select_dtypes(include = ['float64','int64'])



In [11]:
# Instantiate OneHotEncoder
ohe = OneHotEncoder(drop='first', sparse=False)

# Fit and transform the categorical columns
cat_processed = ohe.fit_transform(cat_df)

# Convert the one-hot-encoded array back to a DataFrame
cat_processed_df = pd.DataFrame(cat_processed, columns=ohe.get_feature_names_out(cat_df.columns))


/Users/samuelmendez/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
# scale data

# instanciate scaler
scaler = StandardScaler()

# fit scaler
num_data_scaled = scaler.fit_transform(num_df)




In [13]:
# convert scaled numerical data to dataframe
num_scaled_df = pd.DataFrame(num_data_scaled, columns = num_df.columns)

In [14]:
# merge the two processed data frames
model_data = cat_processed_df.merge(num_scaled_df, left_index=True, right_index=True)

In [18]:
complete_df_columns = ['id','data_id', 'sales']


# merge id and data_id columns
model_df = model_data.merge(complete_df[complete_df_columns],left_index=True, right_index=True)

In [20]:
model_df.columns


Index(['city_Babahoyo', 'city_Cayambe', 'city_Cuenca', 'city_Daule',
       'city_El Carmen', 'city_Esmeraldas', 'city_Guaranda', 'city_Guayaquil',
       'city_Ibarra', 'city_Latacunga', 'city_Libertad', 'city_Loja',
       'city_Machala', 'city_Manta', 'city_Playas', 'city_Puyo',
       'city_Quevedo', 'city_Quito', 'city_Riobamba', 'city_Salinas',
       'city_Santo Domingo', 'state_Bolivar', 'state_Chimborazo',
       'state_Cotopaxi', 'state_El Oro', 'state_Esmeraldas', 'state_Guayas',
       'state_Imbabura', 'state_Loja', 'state_Los Rios', 'state_Manabi',
       'state_Pastaza', 'state_Pichincha', 'state_Santa Elena',
       'state_Santo Domingo de los Tsachilas', 'state_Tungurahua', 'type_B',
       'type_C', 'type_D', 'type_E', 'sale_cat_apparel', 'sale_cat_food_bev',
       'sale_cat_household', 'sale_cat_leisure', 'sale_cat_personal_home',
       'sale_cat_specialty', 'onpromotion', 'dcoilwtico', 'cluster',
       'transactions', 'month', 'day', 'year', 'id', 'data_id', 'sal

In [21]:
model_df.to_csv('model_df.csv')